In [1]:
from cdcqr.data.glassnode.utils import get_glassnode_data, quantile_info, url_parser

2022-01-27 15:12:17,558 - virtualbook - INFO - Table trades created successfully


logging to C:\core\logs\ct.log


In [2]:
from datetime import datetime as dt
from datetime import timedelta
import pandas as pd
import numpy as np
import vectorbt as vbt
import seaborn as sns
from cdcqr.common.utils import camel_case2snake_case
import matplotlib.pyplot as plt
from cdcqr.analytics.utils import ts_eda
from importlib import reload
from cdcqr.common.utils import load_df
%matplotlib inline

import sys
sys.path.append('/core/github/cryptoderiv-quant/')

from ct.utils import plot2, rollingcorr
from cdcqr.common.config import GLASSNODE_API_KEY
API_KEY = GLASSNODE_API_KEY
from ct.utils import glassnode

In [3]:
import cdcqr.analytics.utils as autil 
reload(autil)

<module 'cdcqr.analytics.utils' from 'c:\\core\\github\\cdcqr\\cdcqr\\analytics\\utils.py'>

In [4]:
#dfbtc = pd.read_pickle('/core/tmp/btc_price_20211001_20211205_10Min.pkl')
#dfeth = pd.read_pickle('/core/tmp/btc_price_20211001_20211205_10Min.pkl')
dfbtc = load_df('BTCUSD_PERP@binance').resample_pv('10Min')
dfeth = load_df('ETHUSD_PERP@binance').resample_pv('10Min')

In [5]:
df_underly = {}
df_underly['BTC'] = dfbtc
df_underly['ETH'] = dfeth


In [6]:
circulating_supply = {}
circulating_supply['BTC'] = 18900625
circulating_supply['ETH'] = 118935388

### transactions_features_exchange_activity

In [7]:
'derivatives.FuturesFundingRatePerpetual'
category = 'derivatives'
features = ['FuturesFundingRatePerpetual']
freq='10min'


In [ ]:
for feature in features:
    for underly in ['BTC','ETH']:
        
        start_time = dt.now()-timedelta(hours=9000)
        url1 = 'metrics~{}~{}'.format(category, camel_case2snake_case(feature))
        print(url1, underly, freq, start_time)
        dff = glassnode(url1,a=underly,i=freq,s=start_time,price=False)

        common_idx = dff.index.intersection(dfbtc.index)
        dff = dff.reindex(index = common_idx)
        dfy = df_underly[underly]
        dfy = dfy.reindex(index = common_idx)
        dff[underly] = dfy['c']
        dff[feature_name] = dff['v']
        dff = dff[[feature_name,underly]]
        
        dff['f_n1'] = dff[feature_name]
        #_ = autil.ts_eda(dff, feature_name, 'f_n1', underly)
        

### transactions_features_miner_activity

In [10]:
feature_name = 'TransfersVolumeMinersToExchanges'
underly = 'BTC'
url = 'a={}&category={}&m=transactions.{}'.format(underly, category, feature_name)
att_dict = url_parser(url)
category = att_dict['category']
feature = att_dict['feature']
underly = att_dict['a']

freq='1h'
start_time = dt.now()-timedelta(hours=1000)

dff = get_glassnode_data(category, feature, underly, freq, start_time, API_KEY)

TypeError: glassnode() got an unexpected keyword argument 'API_KEY'

In [ ]:
transactions_features_miner_activity = ['TransfersVolumeMinersToExchanges', 
                                        'TransfersVolumeToMinersSum', 
                                        'TransfersVolumeFromMinersSum', 
                                        'TransfersVolumeMinersNet', 
                                        'TransfersFromMinersCount', 
                                        'TransfersToMinersCount']

# transactions_features_miner_activity = ['TransfersVolumeMinersToExchanges', ]

for feature_name in transactions_features_exchange_activity:
    for underly in ['BTC','ETH']:
        url = 'a={}&category={}&m=transactions.{}'.format(underly, category, feature_name)
        att_dict = url_parser(url)
        category = att_dict['category']
        feature = att_dict['feature']
        underly = att_dict['a']

        freq='1h'
        start_time = dt.now()-timedelta(hours=8760)
        u_time = dt.now()-timedelta(hours=1)
        #dff = get_glassnode_data(category, feature, underly, freq, start_time,  API_KEY)
        url1 = 'metrics~{}~{}'.format(category, feature)
        dff = glassnode(url1,a=underly,i=freq,s=start_time,price=False)
        common_idx = dff.index.intersection(dfbtc.index)
        dff = dff.reindex(index = common_idx)
        dfy = df_underly[underly]
        dfy = dfy.reindex(index = common_idx)
        dff[underly] = dfy['c']
        dff[feature_name] = dff['v']
        dff = dff[[feature_name,underly]]
        
        dff['mc'] = dff[underly]*circulating_supply[underly]
        dff['f_n1'] = dff[feature_name]/dff['mc']
        _ = autil.ts_eda(dff, feature_name, 'f_n1', underly)

### OTC Desks

In [ ]:
category = 'OTC'
transactions_features_otc_desks = ['TransfersVolumeToOtcDesksSum', 
                                        'TransfersVolumeFromOtcDesksSum', 
                                        'TransfersToOtcDesksCount', 
                                        'TransfersFromOtcDesksCount', 
                                        ]

for feature_name in transactions_features_otc_desks:
    for underly in ['BTC']:
        url = 'a={}&category={}&m=transactions.{}'.format(underly, category, feature_name)
        att_dict = url_parser(url)
        category = att_dict['category']
        feature = att_dict['feature']
        underly = att_dict['a']

        freq='1h'
        start_time = dt.now()-timedelta(hours=8760)
        u_time = dt.now()-timedelta(hours=1)
        #dff = get_glassnode_data(category, feature, underly, freq, start_time,  API_KEY)
        url1 = 'metrics~{}~{}'.format(category, feature)
        dff = glassnode(url1,a=underly,i=freq,s=start_time,price=False)
        common_idx = dff.index.intersection(dfbtc.index)
        dff = dff.reindex(index = common_idx)
        dfy = df_underly[underly]
        dfy = dfy.reindex(index = common_idx)
        dff[underly] = dfy['c']
        dff[feature_name] = dff['v']
        dff = dff[[feature_name,underly]]
        
        dff['mc'] = dff[underly]*circulating_supply[underly]
        dff['f_n1'] = dff[feature_name]/dff['mc']
        _ = autil.ts_eda(dff, feature_name, 'f_n1', underly)

In [ ]:
feature_name = 'TransfersVolumeToOtcDesksSum'
underly = 'BTC'
category = 'OTC'
for feature_name in transactions_features_otc_desks:
    ts_eda_wrapper(feature_name, underly, category, freq = '1h')

In [ ]:
def ts_eda_wrapper(feature_name, underly, category, freq = '1h'):
    url = 'a={}&category={}&m=transactions.{}'.format(underly, category, feature_name)
    att_dict = url_parser(url)
    category = att_dict['category']
    feature = att_dict['feature']
    underly = att_dict['a']

    start_time = dt.now()-timedelta(hours=8760)
    u_time = dt.now()-timedelta(hours=1)
    #dff = get_glassnode_data(category, feature, underly, freq, start_time,  API_KEY)
    url1 = 'metrics~{}~{}'.format(category, feature)
    dff = glassnode(url1,a=underly,i=freq,s=start_time,price=False)
    common_idx = dff.index.intersection(dfbtc.index)
    dff = dff.reindex(index = common_idx)
    dfy = df_underly[underly]
    dfy = dfy.reindex(index = common_idx)
    dff[underly] = dfy['c']
    dff[feature_name] = dff['v']
    dff = dff[[feature_name,underly]]

    dff['mc'] = dff[underly]*circulating_supply[underly]
    dff['f_n1'] = dff[feature_name]/dff['mc']
    _ = autil.ts_eda(dff, feature_name, 'f_n1', underly)

### transaction count and size

In [ ]:
transactions_count_and_size = [   'Count', 
                                        'Rate', 
                                        'EntityAdjustedCount', 
                                        'SizeSum', 
                                     'SizeMean'
                                        ]


for feature_name in transactions_count_and_size:
    for underly in ['BTC']:
        url = 'a={}&category={}&m=transactions.{}'.format(underly, category, feature_name)
        att_dict = url_parser(url)
        category = att_dict['category']
        feature = att_dict['feature']
        underly = att_dict['a']

        freq='10m'
        start_time = dt.now()-timedelta(hours=1000)

        dff = get_glassnode_data(category, feature, underly, freq, start_time, API_KEY)
        common_idx = dff.index.intersection(dfbtc.index)
        dff = dff.reindex(index = common_idx)
        dfy = df_underly[underly]
        dfy = dfy.reindex(index = common_idx)
        dff[underly] = dfy['c']
        dff[feature_name] = dff['v']
        dff = dff[[feature_name,underly]]
        _ = autil.ts_eda(dff)

### transaction - transfer volume

In [ ]:
transactions_transfer_volume = ['TransfersVolumeSum', 
                                        'TransfersVolumeMean', 
                                        'TransfersVolumeMedian', 
                                        'TransfersVolumeAdjustedSum', 
                               'TransfersVolumeAdjustedMean',
                                'TransfersVolumeAdjustedMedian',
                                'TransfersVolumeEntityAdjustedSum',
                                'TransfersVolumeEntityAdjustedMean',
                                        ]

underly = 'BTC'
category = 'transaction'
for feature_name in transactions_transfer_volume:
    ts_eda_wrapper(feature_name, underly, category, freq = '1h')

In [ ]:
dff

### Volume - UXTO volume

In [ ]:
category = 'Volume'
volume_uxto_volume = ['UtxoCreatedValueSum', 
                                        'UtxoSpentValueMean', 
                     ]

for feature_name in volume_uxto_volume:
    for underly in ['BTC']:
        url = 'a={}&category={}&m=blockchain.{}'.format(underly, category, feature_name)
        att_dict = url_parser(url)
        category = att_dict['category']
        feature = att_dict['feature']
        underly = att_dict['a']

        freq='10m'
        start_time = dt.now()-timedelta(hours=1000)

        dff = get_glassnode_data(category, feature, underly, freq, start_time, api_key=API_KEY)
        common_idx = dff.index.intersection(dfbtc.index)
        dff = dff.reindex(index = common_idx)
        dfy = df_underly[underly]
        dfy = dfy.reindex(index = common_idx)
        dff[underly] = dfy['c']
        dff[feature_name] = dff['v']
        dff = dff[[feature_name,underly]]
        _ = autil.ts_eda(dff)

### Dsitribution - Entity Balance

In [ ]:
category = 'Distribution'
distribution_entity_balance = ['BalanceExchanges', 
                     ]

for feature_name in distribution_entity_balance:
    for underly in ['BTC']:
        url = 'a={}&category={}&m=distribution.{}'.format(underly, category, feature_name)
        att_dict = url_parser(url)
        category = att_dict['category']
        feature = att_dict['feature']
        underly = att_dict['a']

        freq='10m'
        start_time = dt.now()-timedelta(hours=1000)

        dff = get_glassnode_data(category, feature, underly, freq, start_time, API_KEY)
        common_idx = dff.index.intersection(dfbtc.index)
        dff = dff.reindex(index = common_idx)
        dfy = df_underly[underly]
        dfy = dfy.reindex(index = common_idx)
        dff[underly] = dfy['c']
        dff[feature_name] = dff['v']
        dff = dff[[feature_name,underly]]
        _ = autil.ts_eda(dff)